**извлечение признаков из текста на естественном языке**

частотный анализ

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
np.set_printoptions(precision=2) # вывод на печать до 2 знака
import pandas as pd
import re

In [2]:
pd.options.display.max_colwidth = 200  

## тексты

In [3]:
docs = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
 ]

# docs = [
#     'После посадки в аэропорту Нью-Йорка воздушное судно отогнали в отдаленную часть аэропорта',
#     'У белой медведицы Джованны в зоопарке Мюнхена 21 ноября родился детеныш.',
#     'На юге столицы сотрудница полиции выжила после падения с седьмого этажа.',
#     ]

## простая векторизация ( CountVectorizer )

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
count.fit(docs)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [5]:
print( 'словарь:', len(count.vocabulary_), 'слов\n')  
print( sorted( count.vocabulary_.keys() )  )
# count.vocabulary_

словарь: 9 слов

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [6]:
count.transform(docs).todense()

matrix([[0, 1, 1, 1, 0, 0, 1, 0, 1],
        [0, 2, 0, 1, 0, 1, 1, 0, 1],
        [1, 0, 0, 1, 1, 0, 1, 1, 1],
        [0, 1, 1, 1, 0, 0, 1, 0, 1]])

## векторизация словосочетаний ( CountVectorizer )

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(ngram_range=(2,2))
count.fit(docs)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [8]:
print( 'словарь:', len(count.vocabulary_), 'слов\n')  
print( sorted( count.vocabulary_.keys() )  )
# count.vocabulary_

словарь: 13 слов

['and this', 'document is', 'first document', 'is the', 'is this', 'second document', 'the first', 'the second', 'the third', 'third one', 'this document', 'this is', 'this the']


In [9]:
count.transform(docs).todense()

matrix([[0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0],
        [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1]])

## векторизация с хэшированием ( HashingVectorizer )

In [10]:
from sklearn.feature_extraction.text import HashingVectorizer

hashvect = HashingVectorizer(n_features=4, norm=None, alternate_sign=False, lowercase=True, )
hashvect = hashvect.fit(docs)
hashvect.transform(docs).todense()

matrix([[2., 1., 2., 0.],
        [2., 1., 2., 1.],
        [1., 3., 2., 0.],
        [2., 1., 2., 0.]])

##  CountVectorizer + TF

In [11]:
cv = CountVectorizer()
cv.fit(docs)
cv_doc = cv.transform( docs )
cv_doc.todense()

matrix([[0, 1, 1, 1, 0, 0, 1, 0, 1],
        [0, 2, 0, 1, 0, 1, 1, 0, 1],
        [1, 0, 0, 1, 1, 0, 1, 1, 1],
        [0, 1, 1, 1, 0, 0, 1, 0, 1]])

In [12]:
len(cv.vocabulary_)

9

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer

# применяем коэффициент обратной частоты IDF и нормализацию
tf = TfidfTransformer(use_idf=True, norm='l2')
tf.fit( cv_doc )
tf.transform( cv_doc ).todense()

matrix([[0.  , 0.47, 0.58, 0.38, 0.  , 0.  , 0.38, 0.  , 0.38],
        [0.  , 0.69, 0.  , 0.28, 0.  , 0.54, 0.28, 0.  , 0.28],
        [0.51, 0.  , 0.  , 0.27, 0.51, 0.  , 0.27, 0.51, 0.27],
        [0.  , 0.47, 0.58, 0.38, 0.  , 0.  , 0.38, 0.  , 0.38]])

### TfidfVectorizer

In [14]:
# TfidfVectorizer = CountVectorizer + TfidfTransformer

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(use_idf=True, norm='l2')
tf.fit( docs )
tf.transform( docs ).todense()

matrix([[0.  , 0.47, 0.58, 0.38, 0.  , 0.  , 0.38, 0.  , 0.38],
        [0.  , 0.69, 0.  , 0.28, 0.  , 0.54, 0.28, 0.  , 0.28],
        [0.51, 0.  , 0.  , 0.27, 0.51, 0.  , 0.27, 0.51, 0.27],
        [0.  , 0.47, 0.58, 0.38, 0.  , 0.  , 0.38, 0.  , 0.38]])